In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Demographic Data Processing


In [95]:
def remove_minus(x):
    if (x == "-"):
        return "NaN"
    else:
        return x.replace("-", "").replace("+", "").replace(",", "")

In [100]:
#this data is for 11 variables, all census tracts in California, using ACS 2009 5YR estimates 

demog_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/demographic_data/'
#get percentages of each race
race = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B02001_race_clean.csv')
race["Percent White"] = race["Estimate; White alone"]/race["Estimate; Total:"]
race["Percent Black or African African"] = race["Estimate; Black or African American alone"]/race["Estimate; Total:"]
race["Percent Asian"] = race["Estimate; Asian alone"]/race["Estimate; Total:"]


edu = pd.read_csv(demog_data_path+ 'ACS_09_5YR_S1501_bach_clean.csv')
edu = edu.rename({"Percent of population 25 years and over  with Bachelor\'s degree": "Percent of population 25 years and over with Bachelor\'s degree"}, axis=1)
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
edu["Percent of population 25 years and over with Bachelor\'s degree"] = edu["Percent of population 25 years and over with Bachelor\'s degree"].astype(float).dropna()


#get rid of + and ***
hvalue = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25077_hvalue_clean.csv')
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].apply(lambda x: x.replace("+", "").replace("-", "NaN").replace(",", ""))
hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float)

#hvalue["Estimate; Median value (dollars)"] = hvalue["Estimate; Median value (dollars)"].astype(float).dropna()

#add attached and detached and find the %
single = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25024_singlefam_clean.csv')
single["Total Single Family"] = single["Estimate; 1, attached"]+ single["Estimate; 1, detached"]
single["% Single Family"] = single["Total Single Family"] / single["Estimate; Total:"]
#single["Tract"] = (single["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



#find % Hispanic and non hispanic
hispanic = pd.read_csv(demog_data_path+ 'ACS_17_5YR_B03003_hispanic_clean.csv')
hispanic["% Hispanic or Latino"] = hispanic["Estimate; Total: - Hispanic or Latino"]/hispanic["Estimate; Total:"]
hispanic["% not Hispanic or Latino"] = hispanic["Estimate; Total: - Not Hispanic or Latino"]/hispanic["Estimate; Total:"]
#hispanic["Tract"] = (hispanic["Id2"].astype(str)).apply(lambda x: int(x[4:8]))



poverty = pd.read_csv(demog_data_path+ "ACS_17_5YR_S1701_poverty_clean.csv")
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].apply(lambda x: x.replace("+", "").replace("-", "NaN"))
poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"] = poverty["Percent below poverty level; Estimate; Population for whom poverty status is determined"].astype(float).dropna()
# poverty["Tract"] = (poverty["Id2"].astype(str)).apply(lambda x: int(x[4:8]))

units = pd.read_csv(demog_data_path+'ACS_09_5YR_B25001_units_clean.csv')
#units["Tract"] = (units["Id2"].astype(str)).apply(lambda x: int(x[4:8]))


#find % renter occupied
tenure = pd.read_csv(demog_data_path+ 'ACS_09_5YR_B25003_tenure_clean.csv')
tenure["% Renter Occupied"] = tenure["Estimate; Renter occupied"]/tenure["Estimate; Total:"]
tenure["% Owner Occupied"] = tenure["Estimate; Owner occupied"]/tenure["Estimate; Total:"]


income = pd.read_csv(demog_data_path + 'ACS_09_5YR_S1903_income.csv')
income["Median income (dollars); All households"] = income["Median income (dollars); All households"].apply(remove_minus)

income["Median income (dollars); All households"] = income["Median income (dollars); All households"].astype(float)


#### Create Controlled features dataframe

In [91]:
race_units_merged = race.merge(units, how='left', left_on="Id2", right_on="Id2")
print("race and units merged", "length", len(race_units_merged))
#print("race and units merged", "dtypes", race_units_merged.dtypes)

edu_merged = race_units_merged.merge(edu, how='left', left_on="Id2", right_on="Id2")
print("edu merged", "length", len(edu_merged))
#print("edu merged", "dtypes", edu_merged.dtypes)

hispanic_merged = edu_merged.merge(hispanic, how='left', left_on="Id2", right_on="Id2")
print("hispanic merged", "length", len(hispanic_merged))
#print("hispanic merged", "dtypes", hispanic_merged.dtypes)

income_merged = hispanic_merged.merge(income, how='left', left_on="Id2", right_on="Id2")
print("income merged", "length", len(income_merged))
#print("income", "dtypes", income_merged.dtypes)

poverty_merged = income_merged.merge(poverty, how='left', left_on="Id2", right_on="Id2")
print("poverty merged", "length", len(poverty_merged))
#print("poverty", "dtypes", poverty_merged.dtypes)

hvalue_merged = poverty_merged.merge(hvalue, how='left', left_on="Id2", right_on="Id2")
print("hvalue merged", "length", len(hvalue_merged))
#print("hvalue", "dtypes", hvalue_merged.dtypes)

single_merged = hvalue_merged.merge(single, how='left', left_on="Id2", right_on="Id2")
print("single merged", "length", len(single_merged))
#print("singe merged", "dtypes", single_merged.dtypes)


merged = single_merged.merge(tenure, how='left', left_on="Id2", right_on="Id2")
print("tenure merged", "length", len(merged))


race and units merged length 7049
edu merged length 7049
hispanic merged length 7049
income merged length 7049
poverty merged length 7049
hvalue merged length 7049
single merged length 7049
tenure merged length 7049


In [92]:
merged.columns
merged = merged[["Id2", 'Percent of population 25 years and over with Bachelor\'s degree', '% Hispanic or Latino', 
      'Percent below poverty level; Estimate; Population for whom poverty status is determined', 
      '% Single Family', '% Owner Occupied', 'Percent Black or African African', 'Percent White', 'Percent Asian',
      'Estimate; Total Number of Housing Units', 'Estimate; Median value (dollars)', 
        'Median income (dollars); All households']]
merged = merged.rename({'Percent below poverty level; Estimate; Population for whom poverty status is determined': "% below poverty level",
              'Estimate; Total Number of Housing Units': 'Total number of housing units',
              'Estimate; Median value (dollars)': "Median home value",
              'Median income (dollars); All households': "Median income"})
      
merged.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0
1,6001400200,37.6,0.088633,5.5,0.665236,0.657428,0.022640,0.794798,0.069364,932,909500.0,122647.0
2,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0
3,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0
4,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0


#### Get only Bay Area counties

In [109]:
county_codes = ['6001', '6013', '6041', '6055', '6075', '6081', '6085', '6097', '6095']

In [114]:
merged[merged["Id2"].apply(lambda x: str(x)[:4]).isin(county_codes)]

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0
1,6001400200,37.6,0.088633,5.5,0.665236,0.657428,0.022640,0.794798,0.069364,932,909500.0,122647.0
2,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0
3,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0
4,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0
5,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0
6,6001400700,19.4,0.177176,16.0,0.572421,0.395335,0.463215,0.365035,0.095104,2016,511600.0,32471.0
7,6001400800,33.5,0.110906,12.1,0.423123,0.297976,0.378702,0.349085,0.177371,1678,499100.0,37750.0
8,6001400900,28.2,0.120718,15.3,0.492487,0.438356,0.545182,0.413573,0.007499,1198,524600.0,51767.0
9,6001401000,19.0,0.198015,24.9,0.466443,0.423432,0.568758,0.232120,0.143205,2682,415600.0,36875.0


#### Merge Demographic features with neighbors dataframe to contribute CRA eligibility information by tract and exclude CTs without oppositely coded neighbors

Universe: all of the CTs in the 9 county bay area that had loan activity have oppositely coded neighbors



In [134]:
data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/neighbors/'
cra_elig_neighbors = pd.read_csv(data_path+ 'boolean_tracts_all_2009.csv')
cra_elig_neighbors.head()

,Tract,Year,CRA Eligible
0,6001433700,2009,eligible
1,6001435101,2009,not eligible
2,6001435101,2009,not eligible
3,6001435101,2009,not eligible
4,6001435101,2009,not eligible


In [167]:
demog_with_cra = pd.merge(merged, cra_elig_neighbors, how='right', left_on='Id2', right_on='Tract')
#.drop("Id2", axis=1) 
#i think that this is supposed to
#be a right merge because the demographics dataframe has all the census tracts, while the boolean neighbors one only
#has the census tracts in the bay area that have neighbors
demog_with_cra = demog_with_cra.drop(["Tract", "Year"], axis=1)
len(demog_with_cra)


1228

#### Get total number and percentages of loans data from HMDA files
This is currently for 2009, but next step is to do it for the average of the recovery period 2013-2017


Universe: all of the CTs in the 9 county bay area that had loan activity during 2009

In [291]:
filepath = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data/'
Alameda = pd.read_csv(filepath+'Alameda_2009_parsed.csv')
Alameda["Geoid"] = Alameda["Tract"].apply(lambda x: int(x*100 + 6001000000))

ContraCosta = pd.read_csv(filepath+'ContraCosta_2009_parsed.csv')
ContraCosta["Geoid"] = ContraCosta["Tract"].apply(lambda x: int(x*100 + 6013000000))

Napa = pd.read_csv(filepath+'Napa_2009_parsed.csv')
Napa["Geoid"] = Napa["Tract"].apply(lambda x: int(x*100 + 6055000000))

Marin = pd.read_csv(filepath+'Marin_2009_parsed.csv')
Marin["Geoid"] = Marin["Tract"].apply(lambda x: int(x*100 + 6041000000))

SanMateo = pd.read_csv(filepath+'SanMateo_2009_parsed.csv')
SanMateo["Geoid"] = SanMateo["Tract"].apply(lambda x: int(x*100 + 6081000000))

SanFrancisco = pd.read_csv(filepath+'SanFrancisco_2009_parsed.csv')
SanFrancisco["Geoid"] = SanFrancisco["Tract"].apply(lambda x: int(x*100 + 6075000000))

Solano = pd.read_csv(filepath+'Solano_2009_parsed.csv')
Solano["Geoid"] = Solano["Tract"].apply(lambda x: int(x*100 + 6095000000))

SantaClara = pd.read_csv(filepath+'SantaClara_2009_parsed.csv')
SantaClara["Geoid"] = SantaClara["Tract"].apply(lambda x: int(x*100 + 6085000000))

Sonoma = pd.read_csv(filepath+'Sonoma_2009_parsed.csv')
Sonoma["Geoid"] = Sonoma["Tract"].apply(lambda x: int(x*100 + 6097000000))

counties = [Alameda, ContraCosta, Napa, Marin, SanMateo, SanFrancisco,Solano,SantaClara, Sonoma]

bay_counties_loans = pd.concat(counties).reset_index(drop=True)
bay_counties_loans["Total Loans"] = bay_counties_loans['# HI borrower, LI tract'] + bay_counties_loans['# HI borrower, HI tract']+ bay_counties_loans['# LI borrower, LI tract']
+ bay_counties_loans['# LI borrower, HI tract']
bay_counties_loans = bay_counties_loans.drop(["Tract", "Year"], axis=1)


#### Merge Loans Data with cra_demog features

In [292]:
allvars_2009 = pd.merge(demog_with_cra, bay_counties_loans, how="left", left_on='Id2', right_on='Geoid').drop(['CRA Eligible_y', "Geoid"], axis=1)
allvars_2009['CRA'] = allvars_2009['CRA Eligible_x'].apply(lambda x: 1 if x == 'eligible' else 0)
allvars_2009.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars2009.csv")

allvars_2009.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,...,"% HI borrower, LI tract","# HI borrower, LI tract","% HI borrower, HI tract","# HI borrower, HI tract","% LI borrower, LI tract","# LI borrower, LI tract","%LI borrower, HI tract","# LI borrower, HI tract",Total Loans,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,...,0.000000,0.0,0.981818,54.0,0.0,0.0,0.0,0.0,54.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,...,0.000000,0.0,0.906250,58.0,0.0,0.0,0.0,0.0,58.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,...,0.000000,0.0,0.979167,47.0,0.0,0.0,0.0,0.0,47.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,...,0.877193,50.0,0.000000,0.0,0.0,0.0,0.0,0.0,50.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,...,0.870968,27.0,0.000000,0.0,0.0,0.0,0.0,0.0,27.0,1


#### Regression Data Prep

In [293]:
allvars_2009.columns
noinc = ['% HI borrower, LI tract',
       '# HI borrower, LI tract', '% HI borrower, HI tract',
       '# HI borrower, HI tract', '% LI borrower, LI tract',
       '# LI borrower, LI tract', '%LI borrower, HI tract',
       '# LI borrower, HI tract', 'Geoid', 'Total Loans', "CRA Eligible_x", "type", "County"]

X_vars = allvars_2009.loc[:, ~allvars_2009.columns.isin(noinc)]
y = allvars_2009["Total Loans"]
X_vars.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


In [287]:
# np.isnan(X_vars["% Hispanic or Latino"])
# type(np.float64('nan'))
for col in X_vars.columns:
    num = sum(np.isnan(X_vars[col]))
    print(col, num)
    

Id2 0
Percent of population 25 years and over with Bachelor's degree 1
% Hispanic or Latino 205
Percent below poverty level; Estimate; Population for whom poverty status is determined 205
% Single Family 1
% Owner Occupied 1
Percent Black or African African 1
Percent White 1
Percent Asian 1
Estimate; Total Number of Housing Units 0
Estimate; Median value (dollars) 17
Median income (dollars); All households 1
CRA 0


#### Some of the data is Nan, I dealt with this is 3 ways, discuss the best strategy for this with Carolina

Strategy 1: drop all the Nans, lose ~20% of the data

In [288]:
X_vars = X_vars.dropna()
y = y.dropna()
#need to concatenate x with y, dropna and then unconcatenate in order to do this correctly

Strategy 2: infill nans with mean of column

In [267]:
# X_vars = X_vars.fillna(X_vars.mean())
# y = y.fillna(y.mean())
# X_vars.head()

,Id2,Percent of population 25 years and over with Bachelor's degree,% Hispanic or Latino,Percent below poverty level; Estimate; Population for whom poverty status is determined,% Single Family,% Owner Occupied,Percent Black or African African,Percent White,Percent Asian,Estimate; Total Number of Housing Units,Estimate; Median value (dollars),Median income (dollars); All households,CRA
0,6001400100,34.5,0.037780,4.2,0.912439,0.895260,0.041435,0.774373,0.110724,1439,1000000.0,186439.0,0
1,6001400300,32.1,0.075151,7.5,0.478758,0.405179,0.112208,0.763900,0.075544,2801,718100.0,66638.0,0
2,6001400400,44.0,0.094463,9.2,0.555446,0.433809,0.094956,0.733365,0.078143,2020,790500.0,80391.0,0
3,6001400500,28.1,0.103195,9.4,0.444380,0.437722,0.350093,0.467146,0.066241,1735,572000.0,50658.0,1
4,6001400600,51.9,0.085627,10.9,0.632653,0.470756,0.473150,0.383055,0.032220,784,586700.0,39802.0,1


Strategy 3: infill nans with median of column

In [294]:
X_vars = X_vars.fillna(X_vars.median())
y = y.fillna(y.median())


#### Data Normalization

In [276]:
def normalize_columns(data, mean_df, std_df):
    '''
    Input:
      data (data frame): contains only numeric columns
    Output:
      data frame, the same data, except each column is standardized 
      to have 0-mean and unit variance
    '''
    normalized_data=(data-mean_df.mean())/std_df.std()

    return normalized_data

In [277]:
def unnormalize_columns(data, mean_df, std_df):
    unnormalized_data=(data*std_df.std())+mean_df.mean()

    return unnormalized_data

In [295]:
normal_x_vars = normalize_columns(X_vars, X_vars, X_vars)
normal_y = normalize_columns(y, y, y)

In [296]:
from sklearn import linear_model
sk_model = linear_model.LinearRegression()
sk_model.fit(X_vars,y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [297]:
from collections import OrderedDict
linear_coef_dict = {}
for linear_coef, linear_feature in zip(sk_model.coef_, X_vars.columns):
    linear_coef_dict[linear_feature] = linear_coef
    
sorted_coefs_linear = OrderedDict(sorted(linear_coef_dict.items(), key=lambda t: t[1]))



In [298]:
sorted_coefs_linear

OrderedDict([('Percent White', -121.19122273993328),
             ('Percent Asian', -100.27614736284556),
             ('Percent Black or African African', -77.46192339010048),
             ('% Hispanic or Latino', -28.425372581127245),
             ("Percent of population 25 years and over with Bachelor's degree",
              -0.04669430769279886),
             ('Estimate; Median value (dollars)', -2.882409227626487e-05),
             ('Id2', 1.6382949275907035e-08),
             ('Median income (dollars); All households',
              0.0007925644188201028),
             ('Estimate; Total Number of Housing Units', 0.06514345591004385),
             ('Percent below poverty level; Estimate; Population for whom poverty status is determined',
              0.5295014888704302),
             ('CRA', 12.962926280130292),
             ('% Single Family', 46.120519300123846),
             ('% Owner Occupied', 72.07911068913583)])